In [ ]:
!pip install biopython

In [ ]:
from Bio import SeqIO

### Download

In [ ]:
!wget -q https://raw.githubusercontent.com/nextstrain/ncov/master/example_data/sequences.fasta -O ../data/sequences.fasta

### Import

In [ ]:
records = list(SeqIO.parse('../data/sequences.fasta', 'fasta'))

In [ ]:
len(records)